1. 引用我自己寫的 helper function: query_from_db可以直接從DB獲取資料>
2. Import 會用到的 library
3. 定義了python request 連線的 header

In [11]:
# Import the pre-implement db query function

import os, sys
parent_dir = os.path.dirname(os.getcwd())
#os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.append(parent_dir)
from db_func import query_from_db
import html
import re
import requests
from bs4 import BeautifulSoup
import datetime
import time

headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}

##### 將資料print出來的function，方便debug。

In [6]:
def print_util(urls, processor):
    for url in urls:
        try:
            print()
            temp = processor(url[0])
            print('News Url: \n{}\n'.format(url[0]))
            print("Title: {}".format(temp['news_title']))
            print("News: {}".format(temp['news']))
            #for key in temp.keys():
                #print('{}: {}'.format(key.capitalize(), temp[key]))
            print()
            print()
        except Exception as e:
            print()
            print(e)
            print('ERROR URL:')
            print(url)
            print()

### 選出10個在DB之中有最多資料的新聞來源：
```sql
SELECT news_source,COUNT(*) 
FROM news_db.news_rss_feeds
GROUP BY news_source
ORDER BY 2 DESC
LIMIT 11;
```

|  news_source    | COUNT(*) |
|------------------|----------|
| MSN              | 14204    |
| 自由時報         | 4997     |
| 大紀元           | 4394     |
| 經濟日報         | 3589     |
| ~新浪台灣新聞中心~ | ~3323~     |
| PCHOME           | 3213     |
| ETtoday          | 2585     |
| 新頭殼要聞       | 2117     |
| Rti 中央廣播電臺 | 2010     |
| Yahoo!奇摩股市   | 1976     |
| 公視新聞 |1796|  
最後因為新浪新聞有很多url是無效的，因此沒有放入 DB

### 1. 隨機選出50個Yahoo的新聞連結：
在開發時我是確保50個新聞連結都通過，但為了怕notebook篇幅太長，我改成五個來demo
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Yahoo!奇摩股市' ORDER BY RAND()
LIMIT 50;
```

In [8]:
random_yahoo_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source LIKE '%Yahoo%' ORDER BY RAND()
LIMIT 100; """

yahoo_test_urls = query_from_db(random_yahoo_sql)

In [12]:

def yahoo_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1]
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('time')
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.%fZ")
            db_date_format = '%Y-%m-%d %H:%M:%S'
            res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = res
        except Exception as e:
            logging.error(e)
            print(e)
            pass

    article_body_tag = soup.find('article', attrs = {'itemprop':'articleBody'})
    caas_body_tag = soup.find('div', attrs = {'class': 'caas-body'})
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for index, p in enumerate(p_tags):
                if p.get('content'):
                    temp_content.append(p.get_text().strip())
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif caas_body_tag:
        p_tags = caas_body_tag.find_all('p')
        a_tags = caas_body_tag.find_all('a')
        if p_tags:
            for index, p in enumerate(p_tags):
                temp_content.append(html.unescape(p.get_text()).strip())
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    if temp_content:    
        # Capture the description start with 公告 to content
        if title_category[0][:4] == '【公告】':
            prefix = title_category[0]
        else:
            prefix = ''
        content = prefix + '\n'.join(temp_content)#.replace('。 ', '。\n')
        res_dict['news'] = content
        return res_dict
    else:
        return 

In [13]:
print('Yahoo 新聞：')
print_util(yahoo_test_urls.values, yahoo_content_processor)

Yahoo 新聞：

News Url: 
https://tw.news.yahoo.com/%E8%B7%8C%E5%80%92%E5%A7%8A%E5%82%B3%E5%A5%874-%E6%90%AD%E5%85%AC%E8%BB%8A4%E6%AC%A1%E8%B7%8C%E5%80%92%E6%B1%82%E5%84%9F160%E8%90%AC-%E5%8F%B8%E6%A9%9F-%E5%A5%B9%E5%83%8F%E6%89%AF%E4%B8%8D%E6%8E%89%E7%9A%84%E5%8F%A3%E9%A6%99%E7%B3%96-220000091.html

Title: 跌倒姊傳奇4／搭公車4次跌倒求償160萬 司機：她像扯不掉的口香糖
News: 台北市蔡姓女子因她與家人2009年到2012年間，在公車上4度跌倒，先後向4司機提告，共求償160萬元，她也被封為「跌倒姊」，有司機被嚇到自掏腰包、付錢了事，更有人因此放棄方向盤、另謀出路，被告駕駛們至今仍難從夢魘中脫離，直言她像「扯不掉的口香糖」，希望司法別成為有心人士濫訟工具。
本刊調查，蔡女以公車為主要交通工具，她與家人卻幾乎年年跌倒，先是其兒子強強（化名）在2009年時摔到疑似腦震盪，2011年換蔡女跌倒而尾椎受傷，2012年2月29日是蔡女的1歲外孫摔到腦部挫傷，外孫受傷後隔日又換蔡女跌倒而頭手挫傷，她先後向4名司機提告，共求償160萬元。
案件經媒體曝光後引發大眾關注，蔡女在鏡頭前大聲喊冤，強調自己是因司機駕駛不慎導致她與家人受傷，又不願認錯才會提告爭取權益，如今看來全案卻疑點重重。
2009年11月8日晚上7時許，蔡女帶著當時還在讀幼稚園的強強，坐上光華巴士司機王珮璉駕駛的公車，蔡女聲稱司機加速左轉，導致強強從座位上被重拋倒地，孩子身上出現多處瘀青，甚至疑有腦震盪現象，蔡女提告過失傷害外，也要求百萬賠償。
但同車乘客指證歷歷，指強強當時坐姿詭異，臉朝窗外、臉頰靠椅子，腳翹起來放椅子上，雙手抱膝並陷入沉睡，路人都為他捏把冷汗，蔡女卻置若罔聞，檢察官根據乘客供詞和相關跡證，認為司機無駕駛過失，蔡女卻不斷興訟，從過失傷害告到妨害名譽，相關案件直到2016年才結束。
進出法院的心力交瘁在司機身上留下難以擺脫的夢魘，不願具名的被告司機表示，蔡女從刑事告到民事，不放棄任何一個訴訟機會，法

### 2. 隨機選出50個 MSN 的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 50;
```

In [14]:
random_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 100; """
msn_test_urls = query_from_db(random_msn_sql)

In [15]:

def msn_content_processor(url):
    res_dict = {}
    prefix = ['相關報導', '※', '＊', '更多三立新聞網報導', '延伸閱讀', '資料來源', '更多中時電子報精彩報導', '看了這篇文章的人，也']
    pattern = re.compile(r'^{}'.format('|'.join(prefix)))
    
    if url.startswith('https://www.msn.com/zh-tw/video/'):
        return
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        res_dict['news_title'] = title_tag.text.strip()
        
    
    category_tag = soup.find('div', attrs = {'class':'logowrapper'})
    if category_tag:
        _, category = category_tag.get_text().strip().split('\n')
        res_dict['news_category'] = category
        
    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('div', attrs = {'class': 'timeinfo-txt'})
    if time_tag:
        date_time_tag = time_tag.find('time')
        try:
            d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.000Z") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('MSN date error, url: {}'.format(url))
    
    article_body_tag = soup.find('section', attrs = {'itemprop':'articleBody'})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        div_tags = article_body_tag.find_all('div')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                #print(p.get_text())
                # Ignore the image caption
                if p.find('span'):
                    p.span.decompose()
                if p.find('a'):
                    p.a.decompose()
                if p.text.strip():
                    if pattern.match(p.text.strip()):
                        break
                    elif p.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(p.text.strip()))
        elif div_tags:
            for div in div_tags:
                if div.find('span'):
                    div.span.decompose()
                if div.find('a'):
                    div.a.decompose()
                if div.text.strip():
                    if pattern.match(div.text.strip()):
                        break
                    elif div.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(div.text.strip()))
            
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs

    if len(temp_content):
        content = '\n'.join(temp_content)
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict: 
        #content_parser.logger.error('MSN url: {} did not process properly'.format(url))
        print('MSN url: {} did not process properly'.format(url))
        return 
    return res_dict

In [16]:
print('MSN 新聞：')
print_util(msn_test_urls.values, msn_content_processor)

MSN 新聞：

News Url: 
https://www.msn.com/zh-tw/entertainment/news/%E9%81%8E%E5%B9%B4%E7%95%99%E5%8F%B0-%E9%82%B1%E9%8B%92%E6%BE%A4-%E9%BB%83%E7%A7%8B%E7%94%9F-%E9%BB%83%E6%98%8E%E5%BF%97%E4%B8%8D%E5%9B%9E%E9%84%89/ar-BB1cT5Xa?li=BBqiNIf&amp;srcref=rss

Title: 過年留台！ 邱鋒澤、黃秋生、黃明志不回鄉
News: 疫情嚴峻，許多外籍藝人紛紛滯留台灣過年，來自新加坡的藝人邱鋒澤就怕在飛機上會有染疫危險，乾脆留在台灣，還有香港影帝黃秋生也乾脆相約待在台灣工作的黃明志，一起在台灣圍爐過年。
藝人邱鋒澤：「我今年受不了我的親戚，跟我要A4的拍立得，每次都要這麼大張，哪裡有這麼大張。」
用槌子猛敲小牛，希望把2020不順遂，全都敲掉，邱鋒澤今年主持暴紅，穿上了乳牛裝賀新年，但身為新加坡人的他，過年無法回家鄉了。
藝人邱鋒澤：「不會回去因為後來考慮到疫情，然後比如說怕會在飛機上，或機場(感染)我是跟家人討論後，還是先緩一下。」
各國疫情嚴峻，為了確保自己與家人的安全，加上兩地來返隔離期，要接近一個月，乾脆留台灣過年拚事業，工作滿到春節前夕。
藝人邱鋒澤：「我先就是工作到大約，過一天算一天，2月我不知道我行程是什麼，反正到月底的時候，我才會開始規劃吧。」
待在台灣的外國藝人，除了邱鋒澤，還有來自香港的黃秋生，與馬來西亞歌手黃明志。
去年底黃明志特地約黃秋生吃燒烤，臉書發文還說，兩人同病相憐，有家回不去，黃明志也透露回國隔離期太長，直接待在台灣過年，也答應黃秋生要一起圍爐煮火鍋，過年不會孤單，另外香港演員杜汶澤，目前也在台灣。
杜汶澤臉書發出在高鐵上的照片，手拿高鐵模型水，上面似乎還有站務員寫著，香港加油台灣支持你們的留言，杜汶澤待在台灣一個多月，遨遊台灣吃喝玩樂，雖然還未表示會不會回港過年，在寶島的日子看似相當開心。
《TVBS》提醒您：
因應新冠肺炎疫情，疾管署持續加強疫情監測與邊境管制措施，
如有疑似症狀，請撥打：1922專線，或0800-001922，
並依指示配戴口罩儘速就醫，同時主動告知醫師旅遊史及接觸史，以利及時診斷

### 3. 隨機選出50個自由時報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 50;
```

In [17]:
ltn_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 100; """
ltn_test_urls = query_from_db(ltn_msn_sql)

In [18]:
def ltn_content_processor(url):
    postfix = ["""一手掌握經濟脈動
    點我訂閱自由財經Youtube頻道""", 
    """☆民眾如遇同居關係暴力情形，可撥打113保護專線，或向各地方政府家庭暴力防治中心求助☆""", 
    """☆健康新聞不漏接，按讚追蹤粉絲頁☆更多重要醫藥新聞訊息，請上自由健康網""",
    """《自由開講》是一個提供民眾對話的電子論壇，不論是對政治、經濟或社會、文化等新聞議題，有意見想表達、有話不吐不快，都歡迎你熱烈投稿。文長700字內為優，來稿請附真實姓名（必寫。有筆名請另註）、職業、聯絡電話、E-mail帳號。本報有錄取及刪修權，不付稿酬；請勿一稿多投，錄用與否將不另行通知。投稿信箱：LTNTALK@gmail.com""",
    """「武漢肺炎專區」請點此，更多相關訊息，帶您第一手掌握。""",
    """"""]
    res_dict = {}
    #headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
        else:
            try:
                res_dict['news_title'], res_dict['news_category'] = title_tag.string.split('|')
            except:
                pass
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag_1 = soup.find('meta', attrs = {'property': 'article:published_time'})
    time_tag_2 = soup.find('meta', attrs = {'name': 'pubdate'})
    #time_tag_3 = soup.find('span', attrs = {'class': 'time'})
    if time_tag_1:
        try:
            d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error {}, URL: {}'.format(e2, url))

    elif time_tag_2:
        try:
            d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%S+08:00")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error: {}, URL:{}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag2 = soup.find('div', attrs = {'class':'text boxTitle boxText'})
    article_body_tag3 = soup.find('div', attrs = {'class':'text'})
    temp_content = []
    links = []
    links_descs = []

    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag2:
        p_tags = article_body_tag2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag2.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag3:
        p_tags = article_body_tag3.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag3.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs

    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        print('LTN {}, did not process properly'.format(url))
        #content_parser.logger.error('LTN url: {} did not process properly'.format(url))
        return
    return res_dict

In [19]:
print('自由時報')
print_util(ltn_test_urls.values,ltn_content_processor)

自由時報

News Url: 
https://news.ltn.com.tw/news/politics/breakingnews/3419918

Title: 罷捷發言人稱收到北市警「傳票」 批政治迫害
News: 高雄罷免黃捷總部發言人徐尚賢指控1月初收到台北市中正一分局寄的傳票（通知書），在罷捷投票前夕，這是政治迫害。（記者許麗娟攝）
〔記者許麗娟／高雄報導〕高雄罷免黃捷總部發言人徐尚賢，今天出面指控1月初收到台北市中正一分局寄的傳票（通知書），案由是「公共危險、毀棄損壞案件」，要他14日去台北說明和做筆錄，直指這是綠色恐怖、政治迫害。
徐尚賢指出，他當天沒有北上說明，據他了解，應該是去年（11月22日）他參加的反萊豬大遊行，表達政府與民意向背，但現在民進黨政府竟用公共危險、毀損損壞的案子要「栽贓、嫁禍」他，認為政府先欺負人民，再拘捕人民，欺壓、威嚇百姓，這是綠色恐怖、政治迫害，而且在罷免黃捷之際，綠色恐怖動手清算，有何居心？
對於總統蔡英文日前喊話支持黃捷，罷捷團體認為這是政府干預民間的罷免案，由總統帶頭行政不中立，影響台灣的民主，進行極權專制，究竟是中華民國還是北韓？是民主國家還是極權專制國家。
至於罷韓團體指他們罷免黃捷的理由不充分，因為萊豬不是市議員的權限，徐尚賢表示，反萊豬是全台人民關心的事，民意代表要接受人民檢視，黃捷在議會表決瘦肉精零檢出落跑，連人民的基本食安都不顧，留她何用。
下午在鳳山市區將舉行的罷捷大遊行，罷捷團體預估將有1000人以上參與，甚至可能達到2000人。



News Url: 
https://news.ltn.com.tw/news/life/breakingnews/3420129

Title: 轟邱淑媞沒人性！ 胡采蘋：心裡永遠有那道傷疤
News: 前台北市衛生局長邱淑媞（見圖）在臉書上大肆批評指揮中心，引來網友砲轟。（資料照）
〔即時新聞／綜合報導〕衛福部立桃園醫院爆發群聚感染事件，當年做出和平醫院封院的決策者之一、前北市衛生局長邱淑媞邱淑媞在臉書上大肆批評指揮中心，引來網友砲轟。對此，常對時事發表看法的財經網美胡采蘋砲轟邱淑媞沒人性，「你不要再刺傷大家了，我們心裡永遠都有那道傷疤」。
胡采蘋在臉書表示，邱淑媞自己不知道她是多少人心中永遠的痛，為什麼一定要在傷口上灑鹽，「當年（2003年）和平醫院在所有人不

### 4. 隨機選出50個大紀元的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 50;
```

In [20]:
epoch_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 100; """
epoch_test_urls = query_from_db(epoch_test_sql)

In [21]:
def epoch_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ', 1)
        res_dict['news_title'] = title_category[0]
        
    
    category_tag = soup.find('meta', attrs = {'property':'article:section'})
    if category_tag:
        res_dict['news_category'] = str(category_tag['content'])
        
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Epoch date error {}'.format(e2))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag_2 = soup.find('div', attrs = {'class':'art-content'})
    #print(article_body_tag)
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag_2:
        p_tags = article_body_tag_2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag_2.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Epoch url: {} did not process properly'.format(url))

    return res_dict


In [22]:
print('大紀元：')
print_util(epoch_test_urls.values, epoch_content_processor)

大紀元：

News Url: 
https://www.epochtimes.com/b5/18/6/9/n10469975.htm

Title: 魔鬼在統治著我們的世界(15)：經濟篇(下)
News: 編按：「九評編輯部」兩年前的文章在今天引發更多共鳴，特此推薦，以饗讀者。
【大紀元2018年06月09日訊】第九章 經濟篇：魔鬼的誘餌（下）
目 錄
4. 公有制和計劃經濟是反天理的奴役制
1）公有制是魔鬼套在人民脖子上的枷鎖
2）計劃經濟是註定失敗的全能型權力經濟
5. 馬克思主義「剝削論」是顛倒善惡煽動仇恨的謬論
6. 絕對平均主義的毒根是恨和妒嫉
1）建立在恨和妒嫉基礎上的絕對平均主義
2）推廣經濟平權是走向共產主義的踏板
3）工會是共產邪靈安插進自由社會的釘子
7. 「共產主義理想」是引誘人類走向毀滅之途的誘餌
結語：重德才能「富而太平」
聽文章：
（聽更多新聞請至「聽紀元」）
上天造了人，賦予人智慧和體力，也給了人通過付出勞動得到正當物質回報的生活狀態，讓人獲得基本的生活保障。美國《獨立宣言》寫道：「我們認為下面這些真理是不言而喻的：造物者創造了平等的個人，並賦予他們若干不可剝奪的權利，其中包括生命權、自由權和追求幸福的權利。」[1]這些權利自然也包含對財產的擁有權和支配權。
而馬克思在《共產黨宣言》裡明確表示「共產黨人可以把自己的理論概括為一句話：消滅私有制」。[2]這就是要實行公有制；公有制必然要求實行計劃經濟，這種制度在本質上是違背天理、違背人性的奴役制。
美國反共運動先驅、思想家弗萊德‧施瓦茨在《本性難移的共產主義者》裡講了一個故事，說的是一個訪問者在前蘇聯和美國的汽車厰進行的兩段問答。[3]
問：「誰擁有這家工廠？」
工人答：「我們擁有。」
問：「那誰擁有工廠用的這塊地皮呢？」
工人答：「我們擁有。」
問：「誰擁有工廠製造出來的這些汽車呢？」
工人答：「我們擁有。」
在大樓外邊有一個很大的停車場，角落上停有三輛轎車。訪問者問，「誰擁有停車場的那幾輛汽車呢？」
工人答：「我們擁有它們，不過其中一輛歸廠長用、一輛歸黨委書記用、一輛歸祕密警察用。」
同一個訪問者又來到了美國的一個汽車厰，問工人同樣的問題。
問：「誰擁有這家工廠？」
工人答：「亨利‧福特擁有。」
問：「那誰擁有工廠用的這塊地皮呢？」
工人答：「亨利‧福特擁有。」
問：「誰擁有工廠製

### 5. 隨機選出50個經濟日報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 50;
```

In [23]:
udn_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 100; """
udn_test_urls = query_from_db(udn_test_sql)

In [24]:
import re
def udn_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category_lst = title_tag.string.split(' | ', 3)
        res_dict['news_title'] = title_category_lst[0]
        try:
            res_dict['news_category'] = title_category_lst[2]
        except:
            pass
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('div', attrs = {'class': 'shareBar__info--author'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.find('span').text, "%Y-%m-%d %H:%M")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d_temp = re.search('(\d{4}\-\d{2}\-\d{2}\s\d{2}\:\d{2})', time_tag.text).group(0)
                d1 = datetime.datetime.strptime(d_temp, "%Y-%m-%d %H:%M") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('Epoch date error {}'.format(e3))
                try:
                    d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%d %H:%M:%S") 
                    d1 -= datetime.timedelta(hours=8)
                    db_date_format = '%Y-%m-%d %H:%M:%S'
                    date_res = d1.strftime(db_date_format)
                    res_dict['news_published_date'] = date_res
                except Exception as e3:
                    print(e3)
                    content_parser.logger.info('Epoch date error {}'.format(e3))
    article_body_tag = soup.find('div', attrs = {'id':'article_body'})
    content_temp, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.get_text().strip() and p.get_text().strip() != 'facebook':
                    content_temp.append(p.get_text().strip())
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs      
    content = '\n'.join(content_temp).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('udn url: {} did not process properly'.format(url))

    return res_dict


In [25]:
print_util(udn_test_urls.values, udn_content_processor)


News Url: 
https://money.udn.com/money/story/5599/5197098

Title: 川普離開白宮後首度發聲！向記者透露這項計畫
News: 美國前總統川普。法新社facebook
美國前總統川普離開白宮後於22日首度發聲，儘管他依然對未來計畫保持神秘，卻暗示一些行動正在進行中。
綜合外媒報導，川普在佛州棕梠攤川普國際高爾夫球俱樂部用晚餐時，被華盛頓觀察家報（Washington Examiner）記者問到關於未來的計畫，在其助理請記者離開前，川普簡短的回覆「我們會做些什麼，只是時間還沒到」。
川普於20日總統拜登就職典禮前不久抵達他的海湖莊園，隔日被拍到在俱樂部打高爾夫球。目前為止，川普乎沒有透露他卸任後的計畫。



News Url: 
https://money.udn.com/money/story/5599/5191716

Title: 賀拜登就職 文在寅盼攜手維持區域和平繁榮
News: 美國新總統拜登宣誓就職，韓國總統文在寅今天發出賀電，期盼以堅實韓美同盟維持朝鮮半島及區域和平繁榮，也表示希望早日與拜登會面。
韓聯社報導，文在寅於賀電中表示，拜登一直強調的和解與重建訊息引發美國人民共鳴，相信這位已做好準備的總統將實現美國的統合與繁榮。
他也期待拜登在2019冠狀病毒疾病（COVID-19，武漢肺炎）、氣候變遷、經濟危機等國際議題上展現領導風範。
文在寅強調，韓國將做為美國堅實同盟及可信賴的夥伴，陪伴拜登政府這次旅程。希望持續強化韓美同盟，維護朝鮮半島與區域的和平及繁榮。
文在寅也說，希望能在不久後與拜登面對面，開誠布公地對談。
文在寅同時在推特上以英文寫道「美國回來了」（America is back），祝賀拜登就任。



News Url: 
https://money.udn.com/money/story/5607/5188205

Title: 今日台股盤前重點掃描
News: 台股。記者杜建重／攝影facebook
一、鴻海打進Apple Car鏈 成長動能看旺
摩根大通與大和證券兩大外資看多鴻海（2317）電動車布局，直指鴻海已打入Apple Car供應鏈，成為Apple Car底盤、組件和次系統的關鍵供應商...看原文
二、晶圓雙雄爆量 占大盤1/5
半導體市況熱，晶圓代工產能供不應求

### 6. 隨機選出50個新浪台灣新聞中心的新聞連結： (skip this phase, too many error urls)
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 50;
```

In [26]:
sina_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 20; """
sina_test_urls = query_from_db(sina_test_sql)

In [27]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [ ]:
import requests
from itertools import cycle
import traceback
#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

In [ ]:
import random
# This data was created by using the curl method explained above
headers_list = [
    # Firefox 77 Mac
     {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Firefox 77 Windows
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 83 Mac
    {
        "Connection": "keep-alive",
        "DNT": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
        # Chrome 83 Windows 
    {
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9"
    }
]

def sina_content_processor(url):
    res_dict = {}
   # proxy = next(proxy_pool)
    r = requests.get(url, headers = random.choice(headers_list))
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-1].strip()
    
    
    # Sina did not have FB available
    #fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    #if fb_app_tag:
    #    res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']
        
    time_tag = soup.find('meta', attrs = {'name': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Sina date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'id':'article_content'})
    article_body_tag_2 = soup.find('div', attrs = {'class': 'pcont'})
    if article_body_tag:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content
    elif article_body_tag_2:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Sina url: {} did not process properly'.format(url))

    return res_dict


In [ ]:
print_util(sina_test_urls.values, sina_content_processor)

### 7. 隨機選出50個PCHOME的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 50;
```

In [28]:
pchome_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 100; """
pchome_test_urls = query_from_db(pchome_test_sql)

In [29]:

def pchome_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-2].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('time')

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%S") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('PChome date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'calss':'article_text'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        a_tags = article_body_tag.find_all('a')
        if content:
            content = re.sub('(\n)+', '\n', content)
            content = re.sub(r'(相關新聞[\s\S]+)', '', content)
            res_dict['news'] = content 
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PChome url: {} did not process properly'.format(url))

    return res_dict

In [31]:
print_util(pchome_test_urls.values, pchome_content_processor)


News Url: 
https://news.pchome.com.tw/internation/cna/20210119/index-16110477456542518011.html

Title: 中國新一波疫情特徵：無症狀感染者特別多
News: 中國河北、黑龍江、吉林等省近來爆發的武漢肺炎疫情，出現大量的無症狀感染者，其中包括吉林的「一傳百」超級傳播者，成為這波疫情的特徵。且無症狀感染者的傳染性相對較強。
據官方通報，中國18日新增118例2019冠狀病毒疾病（COVID-19，武漢肺炎）確診病例，其中本土病例占106例，吉林、河北、黑龍江又各占43、35、27例，北京1例；此外，無症狀感染者也新增91例，其中80例為本土病例。
環球時報報導，中國每天新增的無症狀感染者居高不下，加上吉林日前出現的「一傳百」超級傳播者也屬無症狀感染者，使得無症狀感染者成為中國這波疫情的特徵及防控重點。
河北省疾控中心對此表示，大量的無症狀感染者，主要來自密切接觸者、疫情調查、暴露人群，以及曾赴疫情傳播區旅行和居住等4類人群。
同時，無症狀感染者也具有傳染性，尤其是處於潛伏期的無症狀感染者，在發病前1至2天和發病初期，傳染性都相對較強。但他們因為沒有任何明顯症狀，很少主動就醫，病毒傳播力卻不遜於確診病例，成為中國這波疫情防控的一大挑戰。
北京佑安醫院感染科副主任醫師李侗表示，無症狀感染者的呼吸道病毒並不一定比有症狀病例少，傳播能力取決於自身病毒量多少及飛沫傳播力。像是吉林的超級傳播者就是一名培訓講師，即使不咳嗽，說話也能散播病毒。
吉林省疾病預防控制中心應急辦公室主任趙慶龍則說，吉林出現黑龍江輸入病例後，第一時間開展大規模篩查，這使吉林的病例都處於感染早期，其中無症狀感染者比較多，但預料會逐漸轉為確診病例。
一名中國匿名專家認為，中國最近頻頻出現無症狀感染者，是因官方透過更及時的檢測，在還沒有發病的早期就發現這些無症狀感染者，是「檢測量大了」，並非感染者絕對人數上升。



News Url: 
https://news.pchome.com.tw/finance/cna_business/20210121/index-16112299782309722003.html

Title: 共享機車GoShare進駐高雄 累計全台6城市投入營運
News: Gogoro旗下的

### 8. 隨機選出50個Ettoday的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 50;
```

In [37]:
ettoday_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 100; """
ettoday_test_urls = query_from_db(ettoday_test_sql)

In [38]:
def ettoday_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    # Ettoday did not put fb page tag on the content
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    #if fb_page_tag:
        #res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Ettoday date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    temp_content = []
    links, links_descs = [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        
        if p_tags:
            for p in p_tags:
                # Ignore the image caption
                if p.find('strong'):
                    continue
                if p.text and p.text[0] != '▲':
                    temp_content.append(p.text.strip())
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip().replace('\u3000', ' '))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Ettoday url: {} did not process properly'.format(url))

    return res_dict

In [39]:
print_util(ettoday_test_urls.values, ettoday_content_processor)


News Url: 
http://feedproxy.google.com/~r/ettoday/global/~3/FovSIWnai9M/1903583.htm

Title: 快訊／馬德里中心傳巨大爆炸聲！滿街瓦礫碎片　建築物幾乎全毀
News: 記者王致凱／綜合報導
西班牙馬德里20日傳出爆炸案，爆炸地點托萊多門附近突然傳出巨大爆炸聲響，但目前還不清楚爆炸原因，也不清楚是否有人員死傷。根據社群網路上流傳的現場畫面，至少有一棟建築物被摧毀。
西班牙《世界報》（El Mundo）報導，當局初步懷疑為瓦斯氣爆，但真正爆炸原因仍有代調查。據悉，發生爆炸的區域內設有安養院和學校，目前消防人員已經將區域完全封鎖。
Acaba de haber una explosión terrible en la Calle Toledo. pic.twitter.com/CggzntAmtQ
Sacan a los trabajadores y ancianos de la residencia de al lado del edificio donde se ha producido la explosión pic.twitter.com/PUokuPuIPx



News Url: 
http://feedproxy.google.com/~r/ettoday/society/~3/uW9Mdmozyi8/1902682.htm

Title: 驚！唐吉訶德凌晨百人「苦等5小時」進場　民眾：1層樓卡30分鐘
News: 記者游宗樺／台北報導
日本「唐吉訶德」於1月19日上午10時開幕，西門町湧現大批人潮等著入場，直到20日凌晨2時許，唐吉訶德店門口仍有上百名民眾等著入場。《Ettoday》記者凌晨於現場直擊，發現店員已在隊伍末端高舉「等候時間還要5小時！」的看板，但仍有不少人想要排隊進店消費。有民眾從店內出來後，直呼裡面太擠，光從1樓排隊等著上樓就花半小時，塞到爆之下只得趕快走逃生門離開。
由於唐吉訶德開放顧客從門口入場後，要在1樓進行排隊，走電扶梯一路直上3樓，再從3樓逛下來，但現場有進行人數容留管制，以至於從1樓走上電扶梯的時間就要耗費約半小時，有民眾表示，因為裡面人數太多，很悶沒有冷氣，雖然在外面排隊不到半小時就進去了，但是人多到沒辦法逛街、走路，想去的層架都是人

### 9. 隨機選出50個Rti 中央廣播電臺的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 50;
```

In [40]:
rti_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 100; """
rti_test_urls = query_from_db(rti_test_sql)

In [45]:
def rti_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    reserved_keywords = ['Rti', '中央廣播電臺', 'Radio Taiwan International']
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ', 1)
        res_dict['news_title'] = title_category[0]
    category_tag = soup.find('div', attrs = {'class':'swiper-wrapper'})
    if category_tag:
        
        res_dict['news_category'] = category_tag.find('a', attrs = {'class':'active'}).get_text().strip()

    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        keyword_lst = [keyword for keyword in keywords_tag['content'].split(',') if keyword not in reserved_keywords]
        res_dict['news_keywords'] = ','.join(keyword_lst)

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if description_tags:
        res_dict['news_description'] = description_tags[1]['content']

    time_tag = soup.find('li', attrs = {'class': 'date'})
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.text.strip(), "時間：%Y-%m-%d %H:%M")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.text.strip(), "時間：%Y-%m-%d %H:%M:%S")
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                #content_parser.logger.info('RTI url: {} date not process properly, error message {}'.format(url, e2))

    article_body_tag = soup.find('article', attrs = {'class' : None})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('RTI url: {} did not process properly'.format(url))
    return res_dict

In [46]:
print_util(rti_test_urls.values, rti_content_processor)


News Url: 
https://www.rti.org.tw/news/view/id/2089660

Title: 誠摯祝賀拜登賀錦麗就職 蘇揆：盼台美關係越來越好
News: 美國第46任總統副總統拜登與賀錦麗就職，行政院長蘇貞昌今天(21日)在行政院會代表台灣政府像拜登與賀錦麗表達誠摯祝賀。蘇揆表示，台美關係不斷有好的進展，外交是內政的延伸，希望美國新政府上任後，台美關係越來越好。
蘇貞昌表示，在蔡英文總統領導下，台美雙方在經貿交流、區域安全等合作越來越緊密，台美高層官員互訪頻繁。他指出，這次中華民國駐美代表蕭美琴獲得正式邀情出席拜登和賀錦麗的就職典禮，更是台美關係42年來的重大突破。
蘇揆表示，美國準國務卿布林肯強調堅決信守對台灣承諾，確保台灣自我防衛能力，蘇貞昌對第一線外交工作人員的努力成果表示肯定與感謝。蘇貞昌說，內政是外交的延伸，台灣在防疫、經濟等領域成績亮眼，這是全體行政團隊的共同努力，希望在美國新政府上任後，台美關係可以越來越好。



News Url: 
https://www.rti.org.tw/news/view/id/2089858

Title: 拜登就職典禮維安功臣 逾150名國民兵確診
News: 要求匿名的美國官員22日表示，部署在華盛頓特區、為拜登總統(Joe Biden)的就職典禮進行維安工作的國民兵中，有150到200人對俗稱武漢肺炎的2019年冠狀病毒疾病(COVID-19)檢測呈現陽性反應。
在前總統川普的支持者6日攻擊國會山莊之後，美國政府在華府祭出了前所未見的安全措施，包括設置附有尖銳鐵絲網的圍籬、以及國民兵進駐的檢查哨等。
這位匿名官員說，確診COVID-19的人數可能還會增加，但相較於過去數天部署在這座城市的逾2.5萬名國民兵來說，這個數字仍只是小型感染。
國民兵發表聲明說，他們不會去談論COVID-19案例，但人員都有遵照美國疾病管制預防中心(CDC)的指導方針，包括在從家鄉出發、以及抵達華盛頓時接受體溫檢測，並進行篩檢問卷調查。
軍方表示，當局已在安排讓數千名士兵返回家園，預料將在未來5到10天內，讓大約1.5萬人離開華盛頓。
此外，約有7千名國民兵預料至少將留到月底，還有約5千人預期會留到3月中旬。不過，這個人數與日期都有可能改變。
根據路透社對公共衛生數據的分析，美國在21日連續第二天通報逾4

### 10. 隨機選出50個公視新聞網的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 50;
```

In [ ]:
pts_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 5; """
pts_test_urls = query_from_db(pts_test_sql)

In [ ]:
def pts_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit(' | ', 1)
        res_dict['news_title'] = title_category[0].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if len(description_tags) > 1:
        res_dict['news_description'] = description_tags[1]['content']
        
    time_tag = soup.find('div', attrs = {'class': 'maintype-wapper hidden-lg hidden-md'})
    if time_tag:
        date_tag = time_tag.find('h2')
        if date_tag:
            try:
                d1 = datetime.datetime.strptime(date_tag.text, "%Y年%m月%d日") 
                #d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e1:
                print(e1)
                content_parser.logger.info('PTS date error {}, URL: {}'.format(e1, url))

    article_body_tag = soup.find('div', attrs = {'class':'article_content'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        if content:
            res_dict['news'] = content
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PTS url: {} did not process properly'.format(url))

    return res_dict

In [ ]:
print_util(pts_test_urls, pts_content_processor)

### 11. 隨機選出50個新頭殼要聞的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 50;
```

In [ ]:
newstalk_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 5; """
newstalk_test_urls = query_from_db(newstalk_test_sql)

In [ ]:
def newstalk_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1]
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('NewsTalk date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    #print(article_body_tag)
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('NewsTalk url: {} did not process properly'.format(url))

    return res_dict

In [ ]:
print_util(newstalk_test_urls, newstalk_content_processor)